In [51]:
require 'daru'
require 'daru_plotting_plotly'
require 'rbplotly'
require 'hg'

class Numeric
  def mili
    self.to_f * 10**(-3)
  end
end

Daru.plotting_library = :plotly

:plotly

# 実験5

In [46]:
['0', '75', '150'].select { |name| File.exists? "data/jikken5_#{name}.csv" }.map do |name|
  h = Daru::DataFrame.from_csv("data/jikken5_#{name}.csv").to_h
  [
    {x: h['Ei'].to_a, y: h['Eo'].to_a, name: "Io=#{name}mA"},
    {x: h['Ei'].to_a, y: h['Vz'].to_a, name: "Io=#{name}mA"}
  ]
end.tap do |arr|
  arr.transpose.zip(['Eo', 'Vz']).each do |trace, kind|
    Plotly::Plot.new(data: trace, layout: { xaxis: { title: 'Ei(V)'}, yaxis: { title: "#{kind}(V)" }, title: kind }).show
  end
end
nil

#<Plotly::Offline::HTML:0x007fb1832b3608 @id="e73b0715-bae8-4a10-95e8-6c1bb79c3ba9", @data=[{:x=>[10, 11, 12, 13, 14, 15, 16, 17, 18], :y=>[9.4, 9.5, 9.53, 9.58, 9.65, 9.62, 9.65, 9.69, 9.7], :name=>"Io=0mA"}, {:x=>[10, 11, 12, 13, 14, 15, 16, 17, 18], :y=>[8.23, 9.2, 9.4, 9.5, 9.52, 9.58, 9.6, 9.61, 9.65], :name=>"Io=75mA"}, {:x=>[10, 11, 12, 13, 14, 15, 16, 17, 18], :y=>[6.82, 7.8, 8.5, 9.54, 9.5, 9.55, 9.6, 9.62, 9.64], :name=>"Io=150mA"}], @layout={:xaxis=>{:title=>"Ei(V)"}, :yaxis=>{:title=>"Eo(V)"}, :title=>"Eo"}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fb182a4ed50 @id="3fd37236-1e51-4a25-a52d-c3b3e56e047f", @data=[{:x=>[10, 11, 12, 13, 14, 15, 16, 17, 18], :y=>[5.56, 5.58, 5.59, 5.59, 5.6, 5.6, 5.61, 5.61, 5.61], :name=>"Io=0mA"}, {:x=>[10, 11, 12, 13, 14, 15, 16, 17, 18], :y=>[5.54, 5.56, 5.57, 5.58, 5.59, 5.59, 5.6, 5.6, 5.61], :name=>"Io=75mA"}, {:x=>[10, 11, 12, 13, 14, 15, 16, 17, 18], :y=>[5.49, 5.53, 5.55, 5.57, 5.58, 5.59, 5.6, 5.6, 5.61], :name=>"Io=150mA"}], @layout={:xaxis=>{:title=>"Ei(V)"}, :yaxis=>{:title=>"Vz(V)"}, :title=>"Vz"}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

# 実験6

In [74]:
Hg::Hg.create do |gen|
  gen.headers :Io, :Vo, :ripple
  gen.body <<BODY
2.mili, 9.72, 24.mili
22.mili, 9.7, 56.mili
42.mili, 9.69, 88.mili
60.mili, 9.65, 120.mili
80.mili, 9.62, 165.mili
98.mili, 9.60, 235.mili
120.mili, 9.51, 920.mili
140.mili, 9.34, 1900.mili
158.mili, 9.13, 2600.mili
180.mili, 8.82, 3300.mili
200.mili, 8.65, 4000.mili
BODY
end.daru_dataframe.tap do |df|
  df.add_vector :ripple_ratio, df.map_rows { |row| row[:ripple].to_f / row[:Vo] * 100 }
end.write_csv('data/jikken6.csv')

In [76]:
Daru::DataFrame.from_csv('data/jikken6.csv').plot(
  x: 'Io', y: 'Vo',
  layout: {
    title: '出力電圧',
    yaxis: { title: '出力電圧(V)' },
    xaxis: { title: '出力電流(A)' }
    }
  ).show
Daru::DataFrame.from_csv('data/jikken6.csv').plot(
  x: 'Io', y: 'ripple_ratio',
  layout: {
    title: 'リップル含有率',
    yaxis: { title: 'リップル含有率(％)' },
    xaxis: { title: '出力電流(A)' }
    }
  ).show

#<Plotly::Offline::HTML:0x007fb18330a2c8 @id="a4b75005-3749-44b8-ad47-052e109b7f48", @data=[{:x=>[0.002, 0.022, 0.042, 0.06, 0.08, 0.098, 0.12, 0.14, 0.158, 0.18, 0.2], :y=>[9.72, 9.7, 9.69, 9.65, 9.62, 9.6, 9.51, 9.34, 9.13, 8.82, 8.65], :name=>"Vo", :type=>:scatter, :mode=>""}], @layout={:xaxis=>{:title=>"出力電流(A)"}, :yaxis=>{:title=>"出力電圧(V)"}, :title=>"出力電圧", :width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fb1819639a0 @id="14b294c4-1b96-4322-ae3e-b6f5544283cb", @data=[{:x=>[0.002, 0.022, 0.042, 0.06, 0.08, 0.098, 0.12, 0.14, 0.158, 0.18, 0.2], :y=>[0.24691358024691357, 0.577319587628866, 0.9081527347781218, 1.2435233160621761, 1.7151767151767152, 2.447916666666667, 9.674027339642482, 20.342612419700217, 28.477546549835704, 37.41496598639456, 46.24277456647399], :name=>"ripple_ratio", :type=>:scatter, :mode=>""}], @layout={:xaxis=>{:title=>"出力電流(A)"}, :yaxis=>{:title=>"リップル含有率(％)"}, :title=>"リップル含有率", :width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

# 実験7

In [85]:
Hg::Hg.create do |gen|
  gen.headers :Vi, :Vo
  gen.body <<BODY
13.01, 9.67
14.03, 9.70
15.01, 9.72
16.01, 9.76
17.0,  9.79
BODY
end.daru_dataframe.write_csv('data/jikken7_0.csv')

In [88]:
Hg::Hg.create do |gen|
  gen.headers :Vi, :Vo
  gen.body <<BODY
13, 9.55
14.01, 9.6
15.0, 9.64
15.99, 9.69
17.0, 9.71
BODY
end.daru_dataframe.write_csv('data/jikken7_75.csv')

In [90]:
Hg::Hg.create do |gen|
  gen.headers :Vi, :Vo
  gen.body <<BODY
12.99, 8.59
14.03, 8.99
15.01, 9.25
16.04, 9.48
17.03, 9.62
BODY
end.daru_dataframe.write_csv('data/jikken7_150.csv')

In [91]:
['0', '75', '150'].select { |name| File.exists? "data/jikken7_#{name}.csv" }.map do |name|
  h = Daru::DataFrame.from_csv("data/jikken7_#{name}.csv").to_h
  {x: h['Vi'].to_a, y: h['Vo'].to_a, name: "Io=#{name}mA"}
end.tap do |trace|
  Plotly::Plot.new(data: trace, layout: { xaxis: { title: 'Vi(V)'}, yaxis: { title: 'Vo(V)' } }).show
end
nil

#<Plotly::Offline::HTML:0x007fb18191acc8 @id="8f99bc30-e4f1-4a82-8740-6c028dd52be3", @data=[{:x=>[13.01, 14.03, 15.01, 16.01, 17.0], :y=>[9.67, 9.7, 9.72, 9.76, 9.79], :name=>"Io=0mA"}, {:x=>[13, 14.01, 15.0, 15.99, 17.0], :y=>[9.55, 9.6, 9.64, 9.69, 9.71], :name=>"Io=75mA"}, {:x=>[12.99, 14.03, 15.01, 16.04, 17.03], :y=>[8.59, 8.99, 9.25, 9.48, 9.62], :name=>"Io=150mA"}], @layout={:xaxis=>{:title=>"Vi(V)"}, :yaxis=>{:title=>"Vo(V)"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>